# Analysis of Results


## Setup

In [ ]:
pip install scikit_posthocs

In [ ]:
import pandas as pd
pd.options.display.float_format = '{:.11}'.format
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
from scipy.stats import friedmanchisquare
import scikit_posthocs as sp
import os

In [ ]:
# Function to perform statistical tests
def statistics(groups, verbose=False):
    num_groups = len(groups)
    results = {}

    if num_groups >= 2:
        # Conduct Friedman test
        friedman_stat, friedman_p_value = friedmanchisquare(*groups)
        results['Friedman p-value'] = friedman_p_value
        results['Friedman statistic'] = friedman_stat
        if verbose:
            print(f"Friedman Test: Statistic={friedman_stat}, p-value={friedman_p_value}")

        # Conduct Nemenyi post-hoc test if Friedman test is significant
        if friedman_p_value < 0.05:
            data = np.array(groups).T
            nemenyi_results = sp.posthoc_nemenyi_friedman(data)
            results['Nemenyi results'] = nemenyi_results
            if verbose:
                print("Nemenyi Post-Hoc Test Results:")
                print(nemenyi_results)
        else:
            if verbose:
                print("Friedman test is not significant; Nemenyi post-hoc test not performed.")
    else:
        raise ValueError("At least two groups are required for comparison.")

    return results

def calculate_means(df):
    # Dictionary to store the mean of each combination
    means_dict = {}

    # Get all column names
    columns = df.columns

    # Iterate over all possible combinations of the columns
    for r in range(1, len(columns) + 1):
        for combo in itertools.combinations(columns, r):
            combo_name = ''.join(combo)
            means_dict[combo_name] = df[list(combo)].mean(axis=1)

In [ ]:
def get_stereotype(responses_path, calc_mean=False, prompts=[1,2,3,4,5,6,7,8,9,10,13,16,17,18,19,20]):
    df = pd.read_csv(responses_path, index_col=['prompt_id'])
    if 'pronoun_sequence_id' in df.columns:
      df.drop(columns=['pronoun_sequence_id'], inplace=True)
    df = df.loc[:, (df.columns.str.contains('neutral'))]
    #df = responses.groupby('prompt_id').mean()
    df = df[df.index.isin(prompts)]
    #df.columns = ['Young vs Neutral', 'Adult vs Neutral', 'Senior vs Neutral']
    # calibration = pd.read_csv(calibration_path, index_col=['prompt_id'])
    # calibration = calibration.loc[:, calibration.columns.str.contains('neutral')]
    if calc_mean:
      df = pd.DataFrame(df.mean(axis=1).values, index=df.index, columns=['Calibration'])
    # calibration = calibration[calibration.index.isin(prompts)]
    # df['Calibration'] = calibration['Calibration']
    return df

In [ ]:
PATH = '/content/drive/MyDrive/Thesis/Datasets'

# Gender

In [ ]:
data_dict = {
    "Career, Education, and Finance": [1, 2, 7, 11, 18, 32, 41],
    "Entertainment, Leisure, and Preferences": [3, 4, 5, 6, 8, 9, 10, 12, 14, 15, 19, 21, 23, 25, 28, 30, 31, 34, 42, 45, 48, 50],
    "Social Interactions and Relationships": [13, 20, 33, 36, 38, 40, 43, 46, 51, 52],
    "Personal Development and Well-being": [16, 17, 22, 24, 26, 27, 29, 35, 37, 39, 44, 47, 49, 53, 54, 55],
    "All" : [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55]
}

In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import friedmanchisquare
import scikit_posthocs as sp

# Define the paths to the CSV files
paths = [
    ['/content/drive/MyDrive/Thesis/Datasets/Gender/Cosine Similarity/gpt-4o-mini-2024-07-18 gender 2024-07-30 - cos_similarity.csv','/content/drive/MyDrive/Thesis/Datasets/Gender/Calibration/Cosine Similarity/gpt-4o-mini-2024-07-18 gender calibration - cos_similarity.csv'],
    ['/content/drive/MyDrive/Thesis/Datasets/Gender/Cosine Similarity/mistral gender 2024-05-02 - cos_similarity.csv', '/content/drive/MyDrive/Thesis/Datasets/Gender/Calibration/Cosine Similarity/mistral gender calibration - cos_similarity.csv'],
    ['/content/drive/MyDrive/Thesis/Datasets/Gender/Cosine Similarity/claude-3 gender 2024-05-02 - cos_similarity.csv', '/content/drive/MyDrive/Thesis/Datasets/Gender/Calibration/Cosine Similarity/claude-3 gender calibration - cos_similarity.csv'],
    ['/content/drive/MyDrive/Thesis/Datasets/Gender/Cosine Similarity/gemini gender 19-07-2024 - cos_similarity.csv', '/content/drive/MyDrive/Thesis/Datasets/Gender/Calibration/Cosine Similarity/llama-2 gender calibration - cos_similarity.csv'],
    ['/content/drive/MyDrive/Thesis/Datasets/Gender/Cosine Similarity/llama-2 gender 2024-05-02 - cos_similarity.csv', '/content/drive/MyDrive/Thesis/Datasets/Gender/Calibration/Cosine Similarity/llama-2 gender calibration - cos_similarity.csv'],
    ['/content/drive/MyDrive/Thesis/Datasets/Gender/Cosine Similarity/llama-3 gender 2024-05-02 - cos_similarity.csv', '/content/drive/MyDrive/Thesis/Datasets/Gender/Calibration/Cosine Similarity/llama-3 gender calibration - cos_similarity.csv'],
    ['/content/drive/MyDrive/Thesis/Datasets/Gender/Cosine Similarity/gemma gender 2024-05-02 - cos_similarity.csv', '/content/drive/MyDrive/Thesis/Datasets/Gender/Calibration/Cosine Similarity/gemma gender calibration - cos_similarity.csv'],
    ['/content/drive/MyDrive/Thesis/Datasets/Gender/Cosine Similarity/yi gender 2024-05-02 - cos_similarity.csv', '/content/drive/MyDrive/Thesis/Datasets/Gender/Calibration/Cosine Similarity/yi gender calibration - cos_similarity.csv']]

# Initialize the main results DataFrame
results_df = pd.DataFrame()

for key in data_dict.keys():
    print(key)
    # Initialize the results DataFrame for this key
    # Loop through each CSV file and calculate statistics
    for path in paths:
        # Extract LLM name from the path
        llm_name = path[0].split('/')[-1].split(' ')[0]
        print(llm_name)

        # Get indices for prompts
        indicies = data_dict.get(key)  # replace with actual function to get indices

        # Read the CSV files
        prompts = get_stereotype(path[0], prompts=indicies)
        calibration = get_stereotype(path[1], calc_mean=True, prompts=indicies)

        # Merge prompts and calibration data
        prompts = pd.merge(prompts, calibration, on='prompt_id', how='inner')

        if len(prompts.columns) >= 2:
            try:
                len(prompts['cos_similarity: male vs neutral'])
                male = 'cos_similarity: male vs neutral'
                female = 'cos_similarity: female vs neutral'
            except:
                male = 'cos_similarity: neutral vs male'
                female = 'cos_similarity: neutral vs female'

            # Prepare data for male vs female vs calibration comparisons
            groups = [prompts[male], prompts[female], prompts['Calibration']]

            # Perform statistical tests
            stat_results = statistics(groups, verbose=False)
            friedman_p = stat_results.get('Friedman p-value')
            nemenyi_results = stat_results.get('Nemenyi results')

            if friedman_p > 0.05:
                p_male_female = '*'
                #p_male_calibration = '*'
                #p_calibration_female = '*'
            else:
                p_male_female = f'{nemenyi_results[0][1]}'[:4]
                #p_male_calibration = f'{nemenyi_results[0][2]}'[:4]
                #p_calibration_female = f'{nemenyi_results[1][2]}'[:4]

            male_mean = prompts[male].mean()
            female_mean = prompts[female].mean()
            calibration_mean = prompts['Calibration'].mean()

            male_female_mean = prompts[[male, female]].mean().mean()
            male_female_calibration_mean = prompts[[male, female ,'Calibration']].mean().mean()


            male_mean = float(f'{male_mean}'[:4])
            female_mean = float(f'{female_mean}'[:4])
            calibration_mean = float(f'{calibration_mean}'[:4])

            male_female_mean = float(f'{male_female_mean}'[:4])
            male_female_calibration_mean = float(f'{male_female_calibration_mean}'[:4])
            print(len(prompts))
            # Append results to the results DataFrame
            new_row = pd.DataFrame({
                'LLM': [llm_name],
                'Stereotype': key,
                'P Value Male-Female': [p_male_female],
                'male_mean': [male_mean],
                'female_mean':[female_mean],
                'calibration_mean':[calibration_mean],
                'male_female_mean':[male_female_mean],
                'male_female_calibration_mean':[male_female_calibration_mean]

            })
            results_df = pd.concat([results_df, new_row], ignore_index=True)

    # Pivot the results DataFrame outside the loop after all results are collected
    if not results_df.empty:
        results_pivot = results_df.pivot(index='Stereotype', columns='LLM', values='P Value Male-Female')
        results_pivot.columns.name = None  # remove the name of the columns for a cleaner DataFrame

# Display the main results DataFrame
print(f'*************{key}*************')
display(results_pivot)


Career, Education, and Finance
gpt-4o-mini-2024-07-18
238
mistral
238
claude-3
238
gemini
238
llama-2
238
llama-3
238
gemma
238
yi
238
Entertainment, Leisure, and Preferences
gpt-4o-mini-2024-07-18
748
mistral
748
claude-3
748
gemini
751
llama-2
748
llama-3
748
gemma
748
yi
748
Social Interactions and Relationships
gpt-4o-mini-2024-07-18
340
mistral
340
claude-3
340
gemini
337
llama-2
340
llama-3
340
gemma
340
yi
340
Personal Development and Well-being
gpt-4o-mini-2024-07-18
544
mistral
544
claude-3
544
gemini
544
llama-2
544
llama-3
544
gemma
544
yi
544
All
gpt-4o-mini-2024-07-18
1870
mistral
1870
claude-3
1870
gemini
1870
llama-2
1870
llama-3
1870
gemma
1870
yi
1870
*************All*************


,claude-3,gemini,gemma,gpt-4o-mini-2024-07-18,llama-2,llama-3,mistral,yi
Stereotype,,,,,,,,
All,0.00,0.78,0.9,0.40,0.30,0.40,0.70,0.45
"Career, Education, and Finance",*,0.9,*,*,0.30,0.9,*,*
"Entertainment, Leisure, and Preferences",0.01,0.9,0.9,0.80,0.41,0.37,0.37,0.9
Personal Development and Well-being,0.03,0.81,0.55,0.50,0.64,0.32,*,0.07
Social Interactions and Relationships,*,0.20,*,*,0.70,0.61,0.00,*


# Ageism

In [ ]:
data_dict = {
    "Dealing with Change":[1,2,7,8],
    "Cognitive and Physical Abilities":[3,4,6,14,13],
    "Emotional Instability":[17,18,19,20,16],
    "Dependent on Else":[5,9,10],
    "All":[1,2,3,4,5,6,7,8,9,10,13,14,16,17,18,19,20]
}

In [ ]:
import pandas as pd

# Define the paths to the CSV files
paths = [
    [PATH + '/Ageism/Cosine Similarity/gpt-4o-mini-2024-07-18 ageism 2024-07-31 - cos_similarity.csv',PATH + '/Ageism/Calibration/Cosine Similarity/gpt-4o-mini-2024-07-18 ageism calibration - cos_similarity.csv'],
    [PATH + '/Ageism/Cosine Similarity/claude-3 ageism 2024-05-02 - cos_similarity.csv', PATH + '/Ageism/Calibration/Cosine Similarity/claude-3 ageism calibration - cos_similarity.csv'],
    [PATH + '/Ageism/Cosine Similarity/gemini ageism 2024-05-02 - cos_similarity.csv', PATH + '/Ageism/Calibration/Cosine Similarity/gemini ageism calibration - cos_similarity.csv'],
    [PATH + '/Ageism/Cosine Similarity/gemma ageism 2024-05-02 - cos_similarity.csv', PATH + '/Ageism/Calibration/Cosine Similarity/gemma ageism calibration - cos_similarity.csv'],
    [PATH + '/Ageism/Cosine Similarity/llama-2 ageism 2024-05-02 - cos_similarity.csv', PATH + '/Ageism/Calibration/Cosine Similarity/llama-2 ageism calibration - cos_similarity.csv'],
    [PATH + '/Ageism/Cosine Similarity/llama-3 ageism 2024-05-02 - cos_similarity.csv', PATH + '/Ageism/Calibration/Cosine Similarity/llama-3 ageism calibration - cos_similarity.csv'],
    [PATH + '/Ageism/Cosine Similarity/mistral ageism 2024-05-02 - cos_similarity.csv', PATH + '/Ageism/Calibration/Cosine Similarity/mistral ageism calibration - cos_similarity.csv'],
    [PATH + '/Ageism/Cosine Similarity/yi ageism 2024-05-02 - cos_similarity.csv', PATH + '/Ageism/Calibration/Cosine Similarity/yi ageism calibration - cos_similarity.csv']

]
# Initialize the main results DataFrame
results_df = pd.DataFrame()

for key in data_dict.keys():
    print(key)
    # Initialize the results DataFrame for this key
    # Loop through each CSV file and calculate statistics
    for path in paths:
        # Extract LLM name from the path
        llm_name = path[0].split('/')[-1].split(' ')[0]
        print(llm_name, f'*************{key}*************')
        # Get indices for prompts
        indicies = data_dict.get(key)  # replace with actual function to get indices

        # Read the CSV files
        prompts = get_stereotype(path[0], prompts=indicies)
        calibration = get_stereotype(path[1], calc_mean=True, prompts=indicies)
        prompts = pd.merge(prompts, calibration, on='prompt_id', how='inner')
        if llm_name == 'gpt-4o-mini-2024-07-18':
          prompts.columns = ['cos_similarity: young vs neutral', 'cos_similarity: adult vs neutral',
       'cos_similarity: senior vs neutral', 'Calibration']
        # Prepare data for senior vs adult vs young comparisons
        print(prompts.columns)
        groups = [prompts['cos_similarity: senior vs neutral'],
                  prompts['cos_similarity: adult vs neutral'],
                  prompts['cos_similarity: young vs neutral'],
                  prompts['Calibration']]

        # Perform statistical tests
        stat_results = statistics(groups, verbose=True)
        friedman_p = stat_results.get('Friedman p-value')
        friedman_stat = stat_results.get('Friedman statistic')
        nemenyi_results = stat_results.get('Nemenyi results')

        if friedman_p > 0.05:
            p_senior_adult = np.nan#'*'
            p_senior_young = np.nan#'*'
            p_young_adult = np.nan#'*'
        else:
            p_senior_adult = float(f'{nemenyi_results[0][1]:.2f}')
            p_senior_young = float(f'{nemenyi_results[0][2]:.2f}')
            p_young_adult = float(f'{nemenyi_results[1][2]:.2f}')


        senior_mean = prompts['cos_similarity: senior vs neutral'].mean()
        adult_mean = prompts['cos_similarity: adult vs neutral'].mean()
        young_mean = prompts['cos_similarity: young vs neutral'].mean()

        senior_adult_mean = prompts[['cos_similarity: senior vs neutral', 'cos_similarity: adult vs neutral']].mean().mean()
        senior_young_mean = prompts[['cos_similarity: senior vs neutral', 'cos_similarity: young vs neutral']].mean().mean()
        adult_young_mean = prompts[['cos_similarity: young vs neutral', 'cos_similarity: adult vs neutral']].mean().mean()
        senior_adult_young_mean = prompts[['cos_similarity: senior vs neutral', 'cos_similarity: adult vs neutral' ,'cos_similarity: young vs neutral']].mean().mean()


        senior_mean = float(f'{senior_mean:.2f}')
        adult_mean = float(f'{adult_mean:.2f}')
        young_mean = float(f'{young_mean:.2f}')

        senior_adult_mean = float(f'{senior_adult_mean:.2f}')
        senior_young_mean = float(f'{senior_young_mean:.2f}')
        adult_young_mean = float(f'{adult_young_mean:.2f}')
        senior_adult_young_mean = float(f'{senior_adult_young_mean:.2f}')

        # Append results to the results DataFrame
        new_row = pd.DataFrame({
            'LLM': [llm_name],
            'Stereotype': key,
            'Senior-Adult': [p_senior_adult],
            'Senior-Young': [p_senior_young],
            'Adult-Young': [p_young_adult],
            'Senior-Adult Mean': [senior_adult_mean],
            'Senior-Young Mean': [senior_young_mean],
            'Adult-Young Mean': [adult_young_mean],
            'Senior-Adult-Young Mean': [senior_adult_young_mean],
            'Senior Mean': [senior_mean],
            'Adult Mean': [adult_mean],
            'Young Mean': [young_mean]
        })
        results_df = pd.concat([results_df, new_row], ignore_index=True)


Dealing with Change
gpt-4o-mini-2024-07-18 *************Dealing with Change*************
Index(['cos_similarity: young vs neutral', 'cos_similarity: adult vs neutral',
       'cos_similarity: senior vs neutral', 'Calibration'],
      dtype='object')
Friedman Test: Statistic=17.909999999999968, p-value=0.00045905397648431887
Nemenyi Post-Hoc Test Results:
              0              1               2               3
0           1.0  0.45571480539             0.9           0.001
1 0.45571480539            1.0   0.75043552732  0.072566617006
2           0.9  0.75043552732             1.0 0.0040777456916
3         0.001 0.072566617006 0.0040777456916             1.0
claude-3 *************Dealing with Change*************
Index(['cos_similarity: young vs neutral', 'cos_similarity: adult vs neutral',
       'cos_similarity: senior vs neutral', 'Calibration'],
      dtype='object')
Friedman Test: Statistic=45.26999999999998, p-value=8.107440228667095e-10
Nemenyi Post-Hoc Test Results:
       

In [ ]:
results_df

,LLM,Stereotype,Senior-Adult,Senior-Young,Adult-Young,Senior-Adult Mean,Senior-Young Mean,Adult-Young Mean,Senior-Adult-Young Mean,Senior Mean,Adult Mean,Young Mean
0,gpt-4o-mini-2024-07-18,Dealing with Change,0.46,0.9,0.75,0.88,0.87,0.88,0.88,0.87,0.89,0.88
1,claude-3,Dealing with Change,0.0,0.19,0.35,0.83,0.83,0.85,0.83,0.81,0.85,0.84
2,gemini,Dealing with Change,0.16,0.75,0.65,0.73,0.73,0.75,0.74,0.71,0.75,0.74
3,gemma,Dealing with Change,0.06,0.06,0.9,0.77,0.78,0.79,0.78,0.75,0.78,0.8
4,llama-2,Dealing with Change,0.04,0.01,0.9,0.8,0.81,0.83,0.81,0.78,0.83,0.83
5,llama-3,Dealing with Change,0.0,0.9,0.01,0.85,0.82,0.84,0.84,0.82,0.87,0.82
6,mistral,Dealing with Change,0.02,0.06,0.9,0.81,0.8,0.83,0.82,0.78,0.84,0.83
7,yi,Dealing with Change,0.07,0.19,0.9,0.79,0.79,0.8,0.79,0.77,0.81,0.8
8,gpt-4o-mini-2024-07-18,Cognitive and Physical Abilities,0.0,0.25,0.05,0.84,0.83,0.86,0.84,0.82,0.87,0.85
9,claude-3,Cognitive and Physical Abilities,0.02,0.55,0.36,0.79,0.76,0.77,0.77,0.78,0.81,0.73


In [ ]:
# Pivot the table
pivot_df = results_df.melt(id_vars=['LLM', 'Stereotype'], var_name='Age Group', value_name='P Value')
pivot_df = pivot_df.pivot_table(index=['Stereotype', 'Age Group'], columns='LLM', values='P Value')

# Flatten the MultiIndex columns
pivot_df.columns.name = None

# Reset the index to move 'Stereotype' and 'Age Group' back to columns
pivot_df.reset_index(inplace=True)

pivot_df

,Stereotype,Age Group,claude-3,gemini,gemma,gpt-4o-mini-2024-07-18,llama-2,llama-3,mistral,yi
0,All,Adult Mean,0.83,0.74,0.76,0.88,0.78,0.81,0.82,0.77
1,All,Adult-Young,0.08,0.9,0.87,0.0,0.9,0.0,0.9,0.08
2,All,Adult-Young Mean,0.81,0.73,0.76,0.87,0.79,0.79,0.81,0.75
3,All,Senior Mean,0.76,0.69,0.69,0.83,0.72,0.73,0.74,0.71
4,All,Senior-Adult,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,All,Senior-Adult Mean,0.8,0.71,0.73,0.85,0.75,0.77,0.78,0.74
6,All,Senior-Adult-Young Mean,0.8,0.72,0.74,0.85,0.76,0.77,0.79,0.74
7,All,Senior-Young,0.0,0.02,0.0,0.0,0.0,0.0,0.0,0.02
8,All,Senior-Young Mean,0.78,0.71,0.73,0.84,0.76,0.76,0.77,0.73
9,All,Young Mean,0.8,0.73,0.76,0.86,0.79,0.78,0.81,0.74


In [ ]:
pivot_df.to_csv('Ageism Bias.csv')

# Ethnicity

In [ ]:
def get_category_for_prompt_id(prompt_id):
    for category, ids in data_dict.items():
        if prompt_id in ids:
            return category
    return "Unknown"  # Return "Unknown" if the prompt_id is not found in any category

def calculate_combinations_mean(df: pd.DataFrame) -> pd.DataFrame:
    """
    Calculate the mean of each possible combination of columns in the DataFrame.

    Parameters:
    df (pd.DataFrame): The DataFrame to calculate means for.

    Returns:
    pd.DataFrame: A DataFrame with the mean values for each column combination.
                  - Index: Combination of column names (joined by ' - ').
                  - 'Mean': Mean value for the combination.
    """
    # Dictionary to store the mean of each combination
    means_dict = {}

    # Get all column names
    columns = df.columns

    # Iterate over all possible combinations of the columns
    for r in range(1, len(columns) + 1):
        for combo in itertools.combinations(columns, r):
            combo_name = ' - '.join(combo)
            means_dict[combo_name] = df[list(combo)].mean(axis=1).mean(axis=0)

    # Convert to DataFrame with a single column 'Mean'
    combinations_mean_df = pd.DataFrame.from_dict(means_dict, orient='index', columns=['Mean'])
    return combinations_mean_df

def statistics(groups: list, verbose: bool = False) -> dict:
    """
    Perform statistical tests (Friedman test and Nemenyi post-hoc test) on the provided groups.

    Parameters:
    groups (list): A list of lists, where each inner list represents a group of data points.
    verbose (bool): If True, print detailed results of the tests. Default is False.

    Returns:
    dict: A dictionary containing the results of the statistical tests.
          - 'friedman': Result of the Friedman test (statistic and p-value).
          - 'nemenyi': Result of the Nemenyi post-hoc test if applicable, otherwise None.
    """
    num_groups = len(groups)
    results = {}

    # Perform tests only if there are at least 2 groups
    if num_groups >= 2:
        # Conduct Friedman test
        friedman_stat, friedman_p_value = friedmanchisquare(*groups)
        results['friedman'] = f"Friedman Test: Statistic={friedman_stat}, p_value={friedman_p_value}"

        # Conduct Nemenyi post-hoc test if Friedman test is significant
        if friedman_p_value < 0.05:
            data = np.array(groups).T
            nemenyi_results = sp.posthoc_nemenyi_friedman(data)
            results['nemenyi'] = nemenyi_results
            # For Gender
              # p_male_female = nemenyi_results[0][2]
              # p_male_Calibration = nemenyi_results[1][2]
              # p_female_Calibration = nemenyi_results[1][2]

            # For Ageism
              # p_young_adult = nemenyi_results[0][1]
              # p_adult_senior = nemenyi_results[1][2]
              # p_senior_young = nemenyi_results[2][0]
              # p_young_Calibration = nemenyi_results[0][3]
              # p_adult_Calibration = nemenyi_results[1][3]
              # p_senior_Calibration = nemenyi_results[2][3]

            # For Ethnicity:
              # p_Caucasian_American_Neutral_American = nemenyi_results[1][0]
              # p_Asian_Neutral_American = nemenyi_results[2][0]
              # p_African_American_Neutral_American = nemenyi_results[3][0]
              # p_Hispanic_Neutral_American = nemenyi_results[4][0]
              # p_Calibration_Neutral_American = nemenyi_results[5][0]
              # p_Asian_Caucasian_American = nemenyi_results[2][1]
              # p_African_American_Caucasian_American = nemenyi_results[3][1]
              # p_Hispanic_Caucasian_American = nemenyi_results[4][1]
              # p_Calibration_Caucasian_American = nemenyi_results[5][1]
              # p_African_American_Asian = nemenyi_results[3][2]
              # p_Hispanic_Asian = nemenyi_results[4][2]
              # p_Calibration_Asian = nemenyi_results[5][2]
              # p_Hispanic_African_American = nemenyi_results[4][3]
              # p_Calibration_African_American = nemenyi_results[5][3]
              # p_Calibration_Hispanic = nemenyi_results[5][4]
            return results

        # If Friedman test is not significant, return None for Nemenyi results
        elif friedman_p_value is not None:
            if verbose:
                print("Friedman test is not significant (p >= 0.05); Nemenyi post-hoc test not performed.")
            results['nemenyi'] = None
            return results
        else:
            raise ValueError("Data shape of content does not match the requirements to perform Friedman.")
    else:
        raise ValueError("At least two groups are required for comparison.")

def homogeneous_groups(type: str, responses_path: str, calibration_path: str, stereotype_group_id: str = None) -> dict:
    """
    Analyze homogeneity within specified demographic groups (gender, ageism, or ethnicity) based on cosine similarity data.

    Parameters:
    type (str): The demographic type to analyze: 'gender', 'ageism', 'ethnicity', etc.
    responses_path (str): Path to the CSV file containing responses with cosine similarity values.
    calibration_path (str): Path to the CSV file containing calibration's cosine similarity values.
    stereotype_group_id (str, optional): If provided, filters responses to include only those with the specified stereotype group ID. Default is None.

    Returns:
    dict: A dictionary containing the results of the statistical analysis.
          - 'friedman': Result of the Friedman test (statistic and p-value).
          - 'nemenyi': Result of the Nemenyi post-hoc test if applicable, otherwise None.
          - 'means': Mean values for each combination of demographic groups.
    """
    # Load responses data from the specified CSV file, setting 'prompt_id' as the index
    responses = pd.read_csv(responses_path, index_col=['prompt_id'])
    responses.drop(columns=['pronoun_sequence_id'], axis=1, inplace=True)


    # Ethnicity Analysis: Consider cosine similarity between various ethnic groups and neutral prompts
    responses = responses[['cos_similarity: neutral vs neutral american',
                            'cos_similarity: neutral vs caucasian american',
                            'cos_similarity: neutral vs asian',
                            'cos_similarity: neutral vs african american',
                            'cos_similarity: neutral vs hispanic',
                            'stereotype_group_id']]

    # Filter by stereotype group ID if provided
    if stereotype_group_id is not None:
        responses = responses[responses.stereotype_group_id.isin([stereotype_group_id])]
    responses = responses.drop(columns=['stereotype_group_id'])
    responses.columns = ['Neutral American', 'Caucasian American', 'Asian', 'African American', 'Hispanic']

    # Load calibration data and calculate the mean across neutral comparisons
    calibration = pd.read_csv(calibration_path, index_col=['prompt_id'])
    calibration = calibration.loc[:, calibration.columns.str.contains('neutral')]
    calibration = calibration.mean(axis=1)
    calibration.columns = 'calibration'
    calibration.name = 'Calibration'

    # Merge responses with the calibration data
    responses = pd.merge(left=responses, right=calibration, left_index=True, right_index=True)
    #display(responses)
    #print(responses.columns)
    # Create a list of groups for statistical analysis (each group corresponds to a column in responses)
    groups = [responses[col] for col in responses.columns]

    # Perform statistical tests and calculate means for combinations of groups
    results = statistics(groups, verbose=False)
    results['means'] = calculate_combinations_mean(responses)

    return results


## GPT-4o

In [ ]:
type = 'ethnicity'
responses_path = PATH + '/Ethnicity/Cosine Similarity/gpt-4o-mini-2024-07-18 ethnicity 2024-08-13 - cos_similarity.csv'
calibration_path = PATH + '/Ethnicity/Calibration/Cosine Similarity/gpt-4o-mini-2024-07-18 ethnicity calibration - cos_similarity.csv'
results_dict = homogeneous_groups(type, responses_path, calibration_path)

In [ ]:
results_dict['friedman'], results_dict['nemenyi']

('Friedman Test: Statistic=152.88498168498154, p_value=3.245696104175729e-31',
              0            1     2     3     4     5
 0          1.0 0.3712605117 0.001 0.001 0.001   0.9
 1 0.3712605117          1.0 0.001 0.001 0.001   0.9
 2        0.001        0.001   1.0   0.9   0.9 0.001
 3        0.001        0.001   0.9   1.0   0.9 0.001
 4        0.001        0.001   0.9   0.9   1.0 0.001
 5          0.9          0.9 0.001 0.001 0.001   1.0)

## Mistral

In [ ]:
type = 'ethnicity'
responses_path = PATH +'/Ethnicity/Cosine Similarity/mistral ethnicity 2024-08-13 - cos_similarity.csv'
calibration_path = PATH +'/Ethnicity/Calibration/Cosine Similarity/mistral ethnicity calibration - cos_similarity.csv'
results_dict = homogeneous_groups(type, responses_path, calibration_path)
results_dict['friedman'], results_dict['nemenyi']

('Friedman Test: Statistic=195.66886446886383, p_value=2.3976664435577985e-40',
                0              1              2              3             4  \
 0            1.0 0.033714640079          0.001          0.001         0.001   
 1 0.033714640079            1.0          0.001          0.001         0.001   
 2          0.001          0.001            1.0 0.087501692907 0.56937984488   
 3          0.001          0.001 0.087501692907            1.0           0.9   
 4          0.001          0.001  0.56937984488            0.9           1.0   
 5            0.9  0.11128785697          0.001          0.001         0.001   
 
               5  
 0           0.9  
 1 0.11128785697  
 2         0.001  
 3         0.001  
 4         0.001  
 5           1.0  )

## Llama-3

In [ ]:
type = 'ethnicity'
responses_path = '/content/drive/MyDrive/Thesis/Datasets/Ethnicity/Cosine Similarity/llama-3 ethnicity 2024-08-13 - cos_similarity.csv'
calibration_path = '/content/drive/MyDrive/Thesis/Datasets/Ethnicity/Calibration/Cosine Similarity/llama-3 ethnicity calibration - cos_similarity.csv'
results_dict = homogeneous_groups(type, responses_path, calibration_path)
results_dict['friedman'], results_dict['nemenyi']

('Friedman Test: Statistic=286.9950865356417, p_value=6.25131048332146e-60',
               0               1               2               3             4  \
 0           1.0           0.001           0.001           0.001         0.001   
 1         0.001             1.0             0.9 0.0011781154202 0.50858342244   
 2         0.001             0.9             1.0 0.0022547535084 0.60254145701   
 3         0.001 0.0011781154202 0.0022547535084             1.0 0.23714168709   
 4         0.001   0.50858342244   0.60254145701   0.23714168709           1.0   
 5 0.25063505399           0.001           0.001           0.001         0.001   
 
               5  
 0 0.25063505399  
 1         0.001  
 2         0.001  
 3         0.001  
 4         0.001  
 5           1.0  )